Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

Make sure to copy the file path of X_full which is train.csv and X_test_full which is test.csv before running the cell
Upload the dataset in your notebook before starting to run all the cells.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('/content/train.csv', index_col='Id')
X_test_full = pd.read_csv('/content/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full_target = X_full.dropna(axis=0, subset=['SalePrice'])
y = X_full_target.SalePrice
X_full_feature = X_full.drop(['SalePrice'], axis=1)

# To keep things simple, we'll use only numerical predictors
X = X_full_feature.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=1)

Use the next code cell to print the first five rows of the data.


In [3]:
X_train.head(20)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
922,90,67.0,8777,5,7,1900,2003,0.0,1084,0,...,0,0,70,0,0,0,0,0,9,2008
521,190,60.0,10800,4,7,1900,2000,0.0,0,0,...,0,220,114,210,0,0,0,0,8,2008
402,20,65.0,8767,7,5,2005,2005,0.0,24,0,...,400,0,0,0,0,0,0,0,7,2006
281,60,82.0,11287,7,6,1989,1989,340.0,421,0,...,575,0,84,0,196,0,0,0,1,2007
1402,60,62.0,7415,6,5,2004,2004,0.0,759,0,...,398,100,75,0,0,0,0,0,4,2008
132,60,NaN,12224,6,5,2000,2000,40.0,695,0,...,390,24,48,0,0,0,0,0,7,2009
809,80,85.0,13400,5,5,1966,1966,1047.0,516,128,...,484,0,0,0,0,0,0,0,6,2006
1250,20,60.0,7200,5,7,1950,1950,0.0,534,96,...,276,0,80,0,0,0,0,0,5,2007
1084,20,80.0,8800,6,6,1964,1964,425.0,553,0,...,461,0,116,0,0,0,0,700,3,2006


You can already see a few missing values in the first several rows.  In the next step, you'll obtain a more comprehensive understanding of the missing values in the dataset.

# Step 1: Preliminary investigation

Run the code cell below without changes.

In [4]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    209
MasVnrArea       8
GarageYrBlt     61
dtype: int64


### Part A

Use the above output to answer the questions below.

In [5]:
# Fill in the line below: How many rows are in the training data?
num_rows = X_train.shape[0]

# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = missing_val_count_by_column[missing_val_count_by_column > 0].shape[0]

# Fill in the line below: How many missing entries are contained in
# all of the training data?
tot_missing = X_train.isnull().sum().sum()

### Part B
Considering your answers above, what do you think is likely the best approach to dealing with the missing values?

Since there are relatively few missing entries in the data (the column with the greatest percentage of missing values is missing less than 20% of its entries), we can expect that dropping columns is unlikely to yield good results. This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

To compare different approaches to dealing with missing values, you'll use the same `score_dataset()` function from the tutorial.  This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Step 2: Drop columns with missing values

In this step, you'll preprocess the data in `X_train` and `X_valid` to remove columns with missing values.  Set the preprocessed DataFrames to `reduced_X_train` and `reduced_X_valid`, respectively.  

In [7]:
missing_cols=X_train.columns[X_train.isnull().any()]
print(missing_cols)

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')


In [8]:
# Fill in the line below: get names of columns with missing values
missing_cols=X_train.columns[X_train.isnull().any()]
# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(missing_cols,axis=1)
reduced_X_valid = X_valid.drop(missing_cols,axis=1)

Run the next code cell without changes to obtain the MAE for this approach.

In [9]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
16367.230342465757


# Step 3: Imputation

### Part A

Use the next code cell to impute missing values with the mean value along each column.  Set the preprocessed DataFrames to `imputed_X_train` and `imputed_X_valid`.  Make sure that the column names match those in `X_train` and `X_valid`.

In [10]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer=SimpleImputer()
imputed_X_train =pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


Run the next code cell without changes to obtain the MAE for this approach.

In [11]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
16657.755650684932


### Part B

Compare the MAE from each approach.  Does anything surprise you about the results?  Why do you think one approach performed better than the other?

#####Write your answer to the above question in a new text box

##Write your answer here : ____

# Step 4: Generate test predictions

In this final step, you'll use any approach of your choosing to deal with missing values.  Once you've preprocessed the training and validation features, you'll train and evaluate a random forest model.  Then, you'll preprocess the test data before generating predictions that can be submitted to the competition!

### Part A

Use the next code cell to preprocess the training and validation data.  Set the preprocessed DataFrames to `final_X_train` and `final_X_valid`.  **You can use any approach of your choosing here!**  in order for this step to be marked as correct, you need only ensure:
- the preprocessed DataFrames have the same number of columns,
- the preprocessed DataFrames have no missing values,
- `final_X_train` and `y_train` have the same number of rows, and
- `final_X_valid` and `y_valid` have the same number of rows.

In [12]:
# Preprocessed training and validation features
final_X_train =imputed_X_train
final_X_valid = imputed_X_valid

Run the next code cell to train and evaluate a random forest model.  (*Note that we don't use the `score_dataset()` function above, because we will soon use the trained model to generate test predictions!*)

In [13]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
16657.755650684932


### Part B

Use the next code cell to preprocess your test data.  Make sure that you use a method that agrees with how you preprocessed the training and validation data, and set the preprocessed test features to `final_X_test`.

Then, use the preprocessed test features and the trained model to generate test predictions in `preds_test`.

In order for this step to be marked correct, you need only ensure:
- the preprocessed test DataFrame has no missing values, and
- `final_X_test` has the same number of rows as `X_test`.

In [15]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(my_imputer.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Run the next code cell without changes to save your results to a CSV file that can be submitted directly to the competition.

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)